In [17]:
import requests
import re
from typing import List, Dict, Any, Optional

In [18]:
# AIRBYTE_BASE_URL = "http://ec2-54-78-209-246.eu-west-1.compute.amazonaws.com/api" # DEV
AIRBYTE_BASE_URL = "http://ec2-52-16-71-181.eu-west-1.compute.amazonaws.com/api" # PROD


In [19]:
def _get_workspaces() -> dict:
    url = f"{AIRBYTE_BASE_URL}/v1/workspaces/list"
    response = requests.post(url)

    if response.status_code == 200:
        return response.json()
    raise ValueError(response.json()["message"])


def _get_workspace_id() -> str:
    workspaces = _get_workspaces().get("workspaces", [])
    if len(workspaces) > 0:
        return workspaces[0].get("workspaceId", None)
    return None


In [20]:
def find_all_connections() -> list:
    payload = {"workspaceId": _get_workspace_id()}
    url = f"{AIRBYTE_BASE_URL}/v1/connections/list"
    response = requests.post(url, json=payload)

    if response.status_code != 200:
        raise ValueError(response.json()["message"])

    connections = response.json().get("connections", [])
    return [connection for connection in connections]

In [21]:
def get_connections_per_report(
    conn_name: str,
    active_conn: bool = True,
) -> List[Dict[str, str]]:
    all_connections = find_all_connections()
    connections_per_report = []

    if active_conn:
        for c in all_connections:
            if c["status"] == "active" and conn_name in c["name"]:
                connections_per_report.append(c)
    else:
        for c in all_connections:
            if conn_name in c["name"]:
                connections_per_report.append(c)

    return connections_per_report

In [22]:
def get_spapi_connections_per_report(
    conn_name: str,
    conn_streams: List[str],
    active_conn: bool = True,
    conn_pattern: Optional[str] = None,
) -> Dict[str, List[Dict[str, Any]]]:
    spapi_connections = get_connections_per_report(conn_name, active_conn)
    CONN_PATTERN = r"^[^:]*: ([0-9A-Za-z_]*) >> .*$"
    connections_per_report = {}

    if conn_pattern:
        CONN_PATTERN = conn_pattern

    for c in spapi_connections:
        try:
            spapi_report = re.findall(CONN_PATTERN, c["name"])[0]
            if spapi_report in conn_streams:
                connections_per_report.setdefault(spapi_report, []).append(c)
        except IndexError:
            print(f"There is no report found for connector name: {c['name']}")
            pass
    return connections_per_report

In [24]:
all_connections = find_all_connections()

In [27]:
print(type(all_connections))
print(len(all_connections))

<class 'list'>
733


In [26]:
all_connections[:3]

[{'connectionId': '0007276a-856f-4a43-928a-1a0643c0d9b8',
  'name': 'Custom Amazon Seller Partner - cherish_baby_care (EU - UK): GET_FBA_INVENTORY_PLANNING_DATA >> S3 heroes-dl-raw-prod Bucket',
  'namespaceDefinition': 'source',
  'namespaceFormat': '${SOURCE_NAMESPACE}',
  'prefix': 'sp_api/p_account_id=CHERISH_BABY_CARE/p_region_id=EU/p_marketplace_id=UK/p_entity_id=',
  'sourceId': '0907934d-6b53-4dcd-a52d-813d3e96e973',
  'destinationId': '0a044861-e642-4460-843c-44f4d20525c7',
  'operationIds': [],
  'syncCatalog': {'streams': [{'stream': {'name': 'GET_FBA_INVENTORY_PLANNING_DATA',
      'jsonSchema': {'type': 'object',
       'title': 'FBA Manage Inventory Health Report',
       '$schema': 'http://json-schema.org/draft-07/schema#',
       'properties': {'sku': {'type': ['null', 'string']},
        'asin': {'type': ['null', 'string']},
        'alert': {'type': ['null', 'string']},
        'fnsku': {'type': ['null', 'string']},
        'currency': {'type': ['null', 'string']},
  

In [67]:
CONN_NAME = "Custom Amazon Seller Partner"
CONN_STREAMS = ["ListFinancialEvents"]

In [68]:
conns = get_connections_per_report(CONN_NAME)

In [56]:
conns[:2]

[{'connectionId': '00309a40-2c8a-4567-a64b-0fa4f1f22c8f',
  'name': 'Custom Amazon Seller Partner - boba (NA - US): ListFinancialEvents >> S3 heroes-dl-raw-dev Bucket',
  'namespaceDefinition': 'source',
  'namespaceFormat': '${SOURCE_NAMESPACE}',
  'prefix': 'sp_api/p_account_id=BOBA/p_region_id=NA/p_marketplace_id=US/p_entity_id=',
  'sourceId': '388fe1bf-bf0f-45ae-8dbf-b795661a5e45',
  'destinationId': '88d526b0-59d4-4acf-b822-1ba8ad12fc30',
  'operationIds': [],
  'syncCatalog': {'streams': [{'stream': {'name': 'ListFinancialEvents',
      'jsonSchema': {'type': 'object',
       'title': 'ListFinancialEvents',
       '$schema': 'http://json-schema.org/draft-07/schema#',
       'properties': {'_metadata': {'type': 'object',
         'properties': {'report_id': {'type': 'string'},
          'request_params': {'type': 'object'}}},
        'RefundEventList': {'type': ['null', 'array'],
         'items': {'type': 'object',
          'properties': {'PostedDate': {'type': 'string',
      

In [123]:
CONNECTIONS = get_spapi_connections_per_report(CONN_NAME, CONN_STREAMS)


There is no report found for connector name: ALEX_TEST Custom Amazon Seller Partner - cradle_star (NA - US) → S3 heroes-dl-raw-dev Bucket
There is no report found for connector name: Custom Amazon Seller Partner - onco (EU - UK) → S3 heroes-dl-raw-dev Bucket


In [121]:
len(CONNECTIONS)
print(CONNECTIONS)


{'ListFinancialEvents': [{'connectionId': '00309a40-2c8a-4567-a64b-0fa4f1f22c8f', 'name': 'Custom Amazon Seller Partner - boba (NA - US): ListFinancialEvents >> S3 heroes-dl-raw-dev Bucket', 'namespaceDefinition': 'source', 'namespaceFormat': '${SOURCE_NAMESPACE}', 'prefix': 'sp_api/p_account_id=BOBA/p_region_id=NA/p_marketplace_id=US/p_entity_id=', 'sourceId': '388fe1bf-bf0f-45ae-8dbf-b795661a5e45', 'destinationId': '88d526b0-59d4-4acf-b822-1ba8ad12fc30', 'operationIds': [], 'syncCatalog': {'streams': [{'stream': {'name': 'ListFinancialEvents', 'jsonSchema': {'type': 'object', 'title': 'ListFinancialEvents', '$schema': 'http://json-schema.org/draft-07/schema#', 'properties': {'_metadata': {'type': 'object', 'properties': {'report_id': {'type': 'string'}, 'request_params': {'type': 'object'}}}, 'RefundEventList': {'type': ['null', 'array'], 'items': {'type': 'object', 'properties': {'PostedDate': {'type': 'string', 'format': 'date-time'}, 'OrderFeeList': {'type': ['null', 'array']}, 'A